# Python librairies need (need to check the version)

In [1]:
import session_info
session_info.show()

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
import random as rd
from tkinter import * 
import openpyxl

# All the path + need to add the excel file for the LCI
path of the folder

one folder for each element

name of the excel file for the LCI

In [15]:
#path folder
path=Path('C:/Users/philippea/Desktop/Impmet_LCI_auto') #TO CHANGE
folder_LCI='Data_LCI'
path_LCI=path / folder_LCI
folder_python='Data_python'
path_python= path / folder_python

file_step1='LCI_modelling.xlsx'
path_step1= path_LCI / file_step1
file_step2='LCI_modelling_step2.xlsx'
path_step2= path_LCI / file_step2
file_step3='LCI_modelling_step3_BW.xlsx'
path_step3= path_LCI / file_step3

file_ei='Database-Overview-for-ecoinvent-v3.9.1.xlsx'
path_ei= path_python / file_ei
sheet_ei='Cut-Off AO'

file_LCI_step0='LCI_Review&Compilation_W_v1.xlsx' #TO CHANGE
path_step0= path_LCI / file_LCI_step0

file_mapping='unit_name_ecoinvent.xlsx'
path_mapping= path_python / file_mapping


# Code: From green to blue

Check that all the sheets with data are named like that 'PRi_XXX' with i the number of the route

Also check if no other sheet have "PR" in their name

In [13]:

doc=pd.ExcelFile(path_step0)
all_sheets=doc.sheet_names#.str.contains('PR')
sheet_names_list = [name for name in all_sheets if "PR" in name] #change with startwith 
sheet_names_list

['PR1_WC_China',
 'PR2_WC_UnitedStates',
 'PR3_WC_Europe',
 'PR4_APT_Europe_Recycling']

In [6]:
list_name_columns_LCI=["Flow\n(As reported by the authors)",	"Type of flow",	"Classification of flows",	"Value",	
                       "Unit\n(As reported by the authors)",	 "Reference flow\n(to which the LCI is reported)",
                       "Associated unit operation\n(as reported by the authors)",	"Order of the unit operation(s) in the process chain", 
                       "Corresponding life cycle stage"]
list_name_add_blue=["Ei - Activity name",  "Elementary flow",	"Ei - Reference product",	"Elementary flow category", "Elementary flow sub-category",
                    "Ei - Location", "Unit", "Modified value to ei unit", "modified value (ref flow BW activity)", "modified value allocated flow"  ,	"Comment on calculations"]


In [21]:


i=0
for sheet in sheet_names_list:
    data_LCI_PRi=pd.read_excel(path_step0,sheet_name=sheet,header=[0])
    columns_need=data_LCI_PRi.loc[:,list_name_columns_LCI].copy()
    columns_need=columns_need.reindex(columns=[*columns_need.columns.tolist(), *list_name_add_blue])
    i+=1
    with pd.ExcelWriter(path_step1,engine='openpyxl',mode='a',if_sheet_exists='replace') as writer:  
        columns_need.to_excel(writer, sheet_name='PR' + str(i) + '_LCI_Modelling',index=False)

# Manual: Need to fill columns:

J (Ei - Activity name), you can use the file Database-Overview-for-ecoinvent-v3.9.1.xlsx, sheet cut-off AO in the folder Data_python

K (Elementary flow), you can use the file  Database-Overview-for-ecoinvent-v3.9.1.xlsx, sheet elementary exchanges in the folder Data_python

O (Ei - location), use the ecoinvent names (maybe create an excel with all the names? )

# Code: Fill column L and check column N and add unit (column O)

Run the next cell if there is no problem in the country attribution you can save it else check the spelling it can also be that the location is to precise for ecoinvent and run again the cell

Pour l'instant seule la feuille PR1 est traitée, pour faire d'autres feuilles changer le numéro en ligne 1 de la cellule en dessous et de la dernière cellule (à modifier)

In [8]:
data_LCI_PRi_model=pd.read_excel(path_step1,sheet_name='PR1_LCI_Modelling',header=[0]) #A MODIFIER SI AUTRE PR



data_LCI_PRi_model_Activity=data_LCI_PRi_model.loc[data_LCI_PRi_model['Ei - Activity name']==data_LCI_PRi_model['Ei - Activity name']]
list_activity=data_LCI_PRi_model_Activity['Ei - Activity name'].drop_duplicates()

dict_unit=pd.read_excel(path_mapping,sheet_name='unit_ei_BW')
dict_unit=dict(dict_unit.values)
ecoinvent_data=pd.read_excel(path_ei,sheet_name=sheet_ei,header=[0])
pb=False
for activity in list_activity:
   
    #find the reference product + unit
    product_name=ecoinvent_data.loc[ecoinvent_data['Activity Name']==activity].loc[:,'Reference Product Name'].drop_duplicates()
    unit=ecoinvent_data.loc[ecoinvent_data['Activity Name']==activity].loc[:,'Unit'].drop_duplicates()
    if len(product_name)>1:
        list_dict_product=dict(product_name)
        # Create an instance of tkinter frame or window
        win = Tk()
        
        # Set the size of the window
        win.geometry("1000x350")
        
        # Define a function to get the output for selected option
        def selection():
           selected = "You selected the option " + str(radio.get())
           label.config(text=selected)
        
        radio = StringVar()
        Label(text="Choose the best option for the activity:" + str(activity), font=('Aerial 14')).pack()
        
        
        for (text, value) in list_dict_product.items(): 
            Radiobutton(win, text = value, variable = radio, 
                value = value,command=selection).pack(side = TOP, ipady = 5) 
        # Define a label widget
        label = Label(win)
        label.pack(pady=20)
        
        exit_button = Button(win, text="Exit", command=win.destroy) 
        exit_button.pack(pady=40) 
        
        win.mainloop()
        # print("error more than 1 product for the activity")
        # break
        product_name=pd.Series(data=radio.get())
    if len(unit)>1:
        list_dict_unit=dict(unit)
        # Create an instance of tkinter frame or window
        win = Tk()
        
        # Set the size of the window
        win.geometry("1000x350")
        
        # Define a function to get the output for selected option
        def selection():
           selected = "You selected the option " + str(radio.get())
           label.config(text=selected)
        
        radio = StringVar()
        Label(text="Choose the best option for the activity:" + str(activity), font=('Aerial 14')).pack()
        
        
        for (text, value) in list_dict_unit.items(): 
            Radiobutton(win, text = value, variable = radio, 
                value = value,command=selection).pack(side = TOP, ipady = 5) 
        # Define a label widget
        label = Label(win)
        label.pack(pady=20)
        
        exit_button = Button(win, text="Exit", command=win.destroy) 
        exit_button.pack(pady=40) 
        
        win.mainloop()
        # print("error more than 1 product for the activity")
        # break
        unit=pd.Series(data=radio.get())
    if len(product_name) >0:
        index=data_LCI_PRi_model.loc[data_LCI_PRi_model['Ei - Activity name']==activity].index
        #data_LCI_PRi_model.loc[data_LCI_PRi_model['Ei - Activity name']==activity].loc[:,'Ei - Reference product']=product_name
        data_LCI_PRi_model.loc[index,'Ei - Reference product']=str(product_name.values[0])
        data_LCI_PRi_model.loc[index,'Unit']=dict_unit[str(unit.values[0])]
        data=data_LCI_PRi_model.loc[(data_LCI_PRi_model['Ei - Activity name']==activity) & (data_LCI_PRi_model['Ei - Reference product']==str(product_name.values[0]))].loc[:,'Ei - Location']
        list_country=ecoinvent_data.loc[(ecoinvent_data['Activity Name']==activity) & (ecoinvent_data['Reference Product Name']==str(product_name.values[0]))].loc[:,'Geography']
        for country in data:
            if country not in list_country.values:
                print( 'problem ' + country + ' ' + activity + ' ' + str(product_name.values[0]))
                pb=True
if pb == False:
    print ('There is no problem in the country attribution')

There is no problem in the country attribution


In [9]:
#fill the category for elementary flow

dict_flow_efcat=pd.read_excel(path_mapping,sheet_name='EI_flow_cat',header=None)#,index_col=0,header=None)
dict_flow_efcat=dict(dict_flow_efcat.values)

for flow_type in dict_flow_efcat:
    data_LCI_PRi_model.loc[data_LCI_PRi_model['Classification of flows']==flow_type,'Elementary flow category']=dict_flow_efcat[flow_type]#data_LCI_PRi_model.loc[data_LCI_PRi_model['Classification of flows']==flow_type]['Elementary flow category'].replace(flow
    


In [10]:
#if no problem save the file else modify the values in the LCI_modelling.xlsx file
with pd.ExcelWriter(path_step2,engine='openpyxl',mode='a',if_sheet_exists='replace') as writer:  
        data_LCI_PRi_model.to_excel(writer, sheet_name='PR' + str(1) + '_LCI_Modelling',index=False) #MODIFY str(1) if another sheet

#  MANUAL fill the modelling step 2 file

File the values column (check the unit and the reference flow for the LCA)

for elementary flow fill the subcompartement where it is needed

 # Code: From blue to BW format

 Comme avant ne fonctionne que pour PR1, si besoin changer sheet_name (à modifier)

In [12]:
data_activity_blue=pd.read_excel(path_step2,sheet_name='PR1_LCI_Modelling') #A MODIFIER SI AUTRE PR




unit_ope_list=list(data_activity_blue.loc[:,'Associated unit operation\n(as reported by the authors)'].drop_duplicates().apply(str))

lines_name_BW=["Activity", "reference product", "location", "amount", "unit", "comment", "Exchanges"]
last_ope=False

columns_name_BW=["name", "reference product",	"location",	"amount",	"unit",	"database",	"type",	"categories",	"uncertainty type",
                 "loc",	"scale", "minimum", "maximum"]

start_row=0

while last_ope!=True:
    win = Tk()
    win.geometry("500x350")
    
    activity=StringVar() # Value saved here
    ref_prod=StringVar()
    location=StringVar()
    amount=StringVar()
    unit=StringVar()
    comment=StringVar()
    exchanges=StringVar()
    
    n=len(unit_ope_list)
    # #for j in range(n):
    var1=IntVar()
    # var2=IntVar()
    
    def search(variable):
      #print(variable.get())
      return (variable.get())
    
    i=1
    Entry(win, width=7, textvariable=activity).grid(column=2, row=i)
    Label(win, text=lines_name_BW[i-1]).grid(column=1, row=i)
    i+=1
    Entry(win, width=7, textvariable=ref_prod).grid(column=2, row=i)
    Label(win, text=lines_name_BW[i-1]).grid(column=1, row=i)
    i+=1
    Entry(win, width=7, textvariable=location).grid(column=2, row=i)
    Label(win, text=lines_name_BW[i-1]).grid(column=1, row=i)
    i+=1
    Entry(win, width=7, textvariable=amount).grid(column=2, row=i)
    Label(win, text=lines_name_BW[i-1]).grid(column=1, row=i)
    i+=1
    Entry(win, width=7, textvariable=unit).grid(column=2, row=i)
    Label(win, text=lines_name_BW[i-1]).grid(column=1, row=i)
    i+=1
    Entry(win, width=7, textvariable=comment).grid(column=2, row=i)
    Label(win, text=lines_name_BW[i-1]).grid(column=1, row=i)
    i+=1
    Entry(win, width=7, textvariable=exchanges).grid(column=2, row=i)
    Label(win, text=lines_name_BW[i-1]).grid(column=1, row=i)
    i+=1
    
    boxes=[]
    for text,value in enumerate(unit_ope_list):
        var = IntVar()
        Checkbutton(win,text=value,variable=var).grid(column=2, row=i)
        i+=1
        boxes.append([value.strip(),var])
    # Checkbutton(win, text=unit_ope_list[0],variable=var1, onvalue=1, offvalue=0).grid(column=2, row=i)
    # i+=1
    Checkbutton(win, text="Is it the last unit operation?",variable=var1).grid(column=2, row=i)
    
    #a continuer à faire pour tous les cas
    
    Button(win, text="Search", command=win.destroy).grid(column=2, row=i+4) #search(activity)
    
    win.mainloop()
    
    act=activity.get()
    ref=ref_prod.get()
    loc=location.get()
    amount_val=amount.get()
    unit_val=unit.get()
    com=comment.get()
    ex=exchanges.get()
    
    last_ope=var1.get()

    df = pd.DataFrame(lines_name_BW)
    df['col2']=[act,ref,loc,amount_val,unit_val,com, ex]
    df=df.rename(columns=df.iloc[0])
    df=df.drop(df.index[0])

    with pd.ExcelWriter(path_step3,engine='openpyxl',mode='a',if_sheet_exists='overlay') as writer:  
        df.to_excel(writer, sheet_name='PR' + str(1) + '_LCI_BW',index=False,header=True,startrow=start_row)#A MODIFIER SI AUTRE PR



        #writer.save()
    start_row+=7

    list_ope=[]
    for el in boxes: #to have the list of operation unit for this activity
        if el[1].get()==1:
            list_ope.append(el[0])

     #add first inventory
    
    #data_activity=pd.read_excel(path_LCI_BW,header=None,index_col=0)
    
    line1=[act,ref,loc,amount_val,unit_val, '', 'production', '' ,'','','','','']
    data_LCI_activity_BW=pd.DataFrame(data=[line1],columns=columns_name_BW)
    
    
    data_selected=data_activity_blue.loc[data_activity_blue['Associated unit operation\n(as reported by the authors)'].isin(list_ope)].copy() # pour selectionner les lines qui ont les étapes demandées 
    #data_activity_blue
    col_need_BW_activity=['Ei - Activity name','Ei - Reference product', 'Ei - Location', "modified value allocated flow",'Unit']
    data_activity_ei_selec=data_selected.loc[:,col_need_BW_activity].dropna(subset=['Ei - Activity name']).reset_index(drop=True).copy()
    
    data_activity_ei_selec.columns=["name", "reference product",	"location",	"amount",	"unit"]
    data_activity_BW=pd.DataFrame(data=data_activity_ei_selec,columns=columns_name_BW)
    data_activity_BW['database']='ecoinvent 3.9.1 cutoff'
    data_activity_BW['type']='technosphere'
    
    col_need_BW_flow=['Elementary flow','Elementary flow category', 'Ei - Location', "modified value allocated flow",'Unit']
    #!! need to change elementary flow category with :: and sub category
    data_flow_selec=data_selected.loc[:,col_need_BW_flow].dropna(subset=['Elementary flow']).reset_index(drop=True).copy()
    data_flow_selec.columns=["name", "categories",	"location",	"amount",	"unit"]
    
    data_flow_BW=pd.DataFrame(data=data_flow_selec,columns=columns_name_BW)
    data_flow_BW['database']='biosphere3'
    data_flow_BW['type']='biosphere'
    
    all_pd=[data_LCI_activity_BW,data_activity_BW,data_flow_BW]
    data_BW=pd.concat(all_pd,ignore_index=True)

    with pd.ExcelWriter(path_step3,engine='openpyxl',mode='a',if_sheet_exists='overlay') as writer:  #, engine='xlsxwriter'
       data_BW.to_excel(writer, sheet_name='PR' + str(1) + '_LCI_BW',index=False,startrow=start_row)#A MODIFIER SI AUTRE PR




    start_row=start_row+len(data_BW)+2

    if last_ope==1:    
        last_ope= True



# END

In [45]:
lines_name_BW=["Activity", "reference product", "location", "amount", "unit", "comment", "Exchanges"]

win = Tk()
win.geometry("500x350")

activity=StringVar() # Value saved here
ref_prod=StringVar()
location=StringVar()
amount=StringVar()
unit=StringVar()
comment=StringVar()
exchanges=StringVar()

n=len(unit_ope_list)
# #for j in range(n):
var1=IntVar()
# var2=IntVar()

def search(variable):
  #print(variable.get())
  return (variable.get())

i=1
Entry(win, width=7, textvariable=activity).grid(column=2, row=i)
Label(win, text=lines_name_BW[i-1]).grid(column=1, row=i)
i+=1
Entry(win, width=7, textvariable=ref_prod).grid(column=2, row=i)
Label(win, text=lines_name_BW[i-1]).grid(column=1, row=i)
i+=1
Entry(win, width=7, textvariable=location).grid(column=2, row=i)
Label(win, text=lines_name_BW[i-1]).grid(column=1, row=i)
i+=1
Entry(win, width=7, textvariable=amount).grid(column=2, row=i)
Label(win, text=lines_name_BW[i-1]).grid(column=1, row=i)
i+=1
Entry(win, width=7, textvariable=unit).grid(column=2, row=i)
Label(win, text=lines_name_BW[i-1]).grid(column=1, row=i)
i+=1
Entry(win, width=7, textvariable=comment).grid(column=2, row=i)
Label(win, text=lines_name_BW[i-1]).grid(column=1, row=i)
i+=1
Entry(win, width=7, textvariable=exchanges).grid(column=2, row=i)
Label(win, text=lines_name_BW[i-1]).grid(column=1, row=i)
i+=1

boxes=[]
for text,value in enumerate(unit_ope_list):
    var = IntVar()
    Checkbutton(win,text=value,variable=var).grid(column=2, row=i)
    i+=1
    boxes.append([value.strip(),var])
# Checkbutton(win, text=unit_ope_list[0],variable=var1, onvalue=1, offvalue=0).grid(column=2, row=i)
# i+=1
Checkbutton(win, text="Is it the last unit operation?",variable=var1).grid(column=2, row=i)

#a continuer à faire pour tous les cas

Button(win, text="Search", command=win.destroy).grid(column=2, row=i+4) #search(activity)

win.mainloop()

act=activity.get()
ref=ref_prod.get()
loc=location.get()
amount_val=amount.get()
unit_val=unit.get()
com=comment.get()
ex=exchanges.get()

last_ope=var1.get()

In [42]:
var1.get()

1

In [97]:
win = Tk()
win.geometry("1000x350")
variable1=StringVar() # Value saved here

def search():
  print(variable1.get())
  return (variable1.get())

Entry(win, width=7, textvariable=variable1).grid(column=2, row=1)
Label(win, text="label").grid(column=1, row=1)

Button(win, text="Search", command=search).grid(column=2, row=13)

win.mainloop()
variable1.get()

zzzzzzz


'zzzzzzz'

In [98]:
variable1.get()

'zzzzzzz'

In [86]:
lines_name_BW=[ "reference product", "location", "amount", "unit", "comment", "Exchanges"]

win = Tk()
win.geometry("1000x350")

i=1
entries=[]
for name in lines_name_BW:
    print(name)
    ent=Entry(win,textvariable=StringVar()).grid(column=2, row=i)
    Label(win, text=name).grid(column=1, row=i)
    #entry.pack()
    entries.append(ent.get())
    i+=1


#Label(win, text="label")
Button(win, text="Search", command=win.destroy).grid(column=2, row=i)

win.mainloop()

reference product


AttributeError: 'NoneType' object has no attribute 'get'

In [83]:
entries

[None, None, None, None, None, None]

In [84]:
contents=[]
for e in entries:
    contents.append(e.get())

contents

AttributeError: 'NoneType' object has no attribute 'get'

In [35]:
#to chose the unit operation need for each activity model in BW format

list_dict_ope=data_activity_blue.loc[:,'Associated unit operation\n(as reported by the authors)'].drop_duplicates()
# Create an instance of tkinter frame or window
win = Tk()

# Set the size of the window
win.geometry("1000x350")

# Define a function to get the output for selected option
def selection():
   selected = "You selected the option " + str(radio.get())
   label.config(text=selected)

radio = StringVar()
Label(text="Choose all the unit operation need for this activity:", font=('Aerial 14')).pack()


for (text, value) in list_dict_ope.items(): 
    print(value)
    Checkbutton(win, text = value, variable = radio, onvalue=1, offvalue=0,command=selection).pack(side = TOP, ipady = 5) 
# Define a label widget
label = Label(win)
label.pack(pady=20)

exit_button = Button(win, text="Exit", command=win.destroy) 
exit_button.pack(pady=40) 

win.mainloop()
# print("error more than 1 product for the activity")
# break
product_name=pd.Series(data=radio.get())

product_name

W ore mining
W concentration
W ore mining + W concentration
APT production
WC production


0    
dtype: object

In [27]:
#lines 1 to 7
lines_name_BW=[ "reference product", "location", "amount", "unit", "comment", "Exchanges"]
df = pd.DataFrame({'Activity':lines_name_BW})

with pd.ExcelWriter('C:/Users/philippea/Desktop/Python/Impmet/LCI_modelling_step3_BW.xlsx',mode='w') as writer:  
    df.to_excel(writer, sheet_name='PR' + str(0) + '_LCI_BW',index=False)
    #writer.save()



In [2]:
columns_name_BW=["name", "reference product",	"location",	"amount",	"unit",	"database",	"type",	"categories",	"uncertainty type",
                 "loc",	"scale", "minimum", "maximum"]



In [8]:
#add first inventory
path_LCI_BW=Path('C:/Users/philippea/Desktop/Python/Impmet')
name_file='LCI_modelling_step3_BW.xlsx'
path_LCI_BW=path_LCI_BW / name_file

data_activity_blue=pd.read_excel("D:/Documents/philippea/OneDrive - BRGM/IMPMET/Python/Data/LCI_modelling_step2.xlsx")

data_activity=pd.read_excel(path_LCI_BW,header=None,index_col=0)

line1=[data_activity.loc['Activity'].values[0],data_activity.loc['reference product'].values[0],data_activity.loc['location'].values[0],
       data_activity.loc['amount'].values[0],data_activity.loc['unit'].values[0], '',' production',' ' ,'','','','','']
data_LCI_activity_BW=pd.DataFrame(data=[line1],columns=columns_name_BW)


data_selected=data_activity_blue.loc[data_activity_blue.loc[:,'Corresponding life cycle stage'].str.contains('Mining')| 
                    data_activity_blue.loc[:,'Corresponding life cycle stage'].str.contains('Concentration')
                    |data_activity_blue.loc[:,'Corresponding life cycle stage'].str.contains('Agg \- M\+C')] # pour selectionner les lines qui ont les étapes demandées 
#data_activity_blue
col_need_BW_activity=['Ei - Activity name','Ei - Reference product', 'Ei - Location', 'Modified value','Unit']
data_activity_ei_selec=data_selected.loc[:,col_need_BW_activity].dropna(subset=['Ei - Activity name']).reset_index(drop=True).copy()

data_activity_ei_selec.columns=["name", "reference product",	"location",	"amount",	"unit"]
data_activity_BW=pd.DataFrame(data=data_activity_ei_selec,columns=columns_name_BW)
data_activity_BW['database']='ecoinvent 3.9.1 cutoff'
data_activity_BW['type']='technosphere'

col_need_BW_flow=['Elementary flow','Elementary flow category', 'Ei - Location', 'Modified value','Unit']
#!! need to change elementary flow category with :: and sub category
data_flow_selec=data_selected.loc[:,col_need_BW_flow].dropna(subset=['Elementary flow']).reset_index(drop=True).copy()
data_flow_selec.columns=["name", "categories",	"location",	"amount",	"unit"]

data_flow_BW=pd.DataFrame(data=data_flow_selec,columns=columns_name_BW)
data_flow_BW['database']='biosphere3'
data_flow_BW['type']='biosphere'

all_pd=[data_LCI_activity_BW,data_activity_BW,data_flow_BW]
data_BW=pd.concat(all_pd,ignore_index=True)
data_BW

,name,reference product,location,amount,unit,database,type,categories,uncertainty type,loc,scale,minimum,maximum
0,[M+C] tungsten ore mining and concentration,tungsten concentrate,CN,1.00,kilogram,,production,,,,,,
1,"market for explosive, tovex","explosive, tovex",GLO,510.00,kg,ecoinvent 3.9.1 cutoff,technosphere,NaN,NaN,NaN,NaN,NaN,NaN
2,"market for steel, low-alloyed","steel, low-alloyed",GLO,680.00,kg,ecoinvent 3.9.1 cutoff,technosphere,NaN,NaN,NaN,NaN,NaN,NaN
3,"particleboard production, uncoated, from virgi...","particleboard, uncoated",RoW,1.77,m3,ecoinvent 3.9.1 cutoff,technosphere,NaN,NaN,NaN,NaN,NaN,NaN
4,Not to be modelled,NaN,NaN,NaN,NaN,ecoinvent 3.9.1 cutoff,technosphere,NaN,NaN,NaN,NaN,NaN,NaN
5,"market for steel, low-alloyed","steel, low-alloyed",GLO,200.00,kg,ecoinvent 3.9.1 cutoff,technosphere,NaN,NaN,NaN,NaN,NaN,NaN
6,market for iron pellet,iron pellet,GLO,150.00,kg,ecoinvent 3.9.1 cutoff,technosphere,NaN,NaN,NaN,NaN,NaN,NaN
7,market for sodium ethyl xanthate,sodium ethyl xanthate,GLO,28.54,kg,ecoinvent 3.9.1 cutoff,technosphere,NaN,NaN,NaN,NaN,NaN,NaN
8,market for light fuel oil,light fuel oil,RoW,8.80,kg,ecoinvent 3.9.1 cutoff,technosphere,NaN,NaN,NaN,NaN,NaN,NaN
9,market for kerosene,kerosene,RoW,53.34,kg,ecoinvent 3.9.1 cutoff,technosphere,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
with pd.ExcelWriter('C:/Users/philippea/Desktop/Python/Impmet/LCI_modelling_step3_BW.xlsx',mode='a', engine='openpyxl',if_sheet_exists='overlay') as writer:  #, engine='xlsxwriter'
       data_BW.to_excel(writer, sheet_name='PR' + str(0) + '_LCI_BW',index=False,startrow=8)

In [47]:
c=data_activity_blue.loc[:,'Corresponding life cycle stage']
c.str.contains("""Mining (M)""")#| data_activity_blue.loc[:,'Corresponding life cycle stage'].str.contains('Concentration (C)') |data_activity_blue.loc[:,'Corresponding life cycle stage'].str.contains('Agg - M+C')

C:\Users\philippea\AppData\Local\Temp\ipykernel_8592\4039961195.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  c.str.contains("""Mining (M)""")#| data_activity_blue.loc[:,'Corresponding life cycle stage'].str.contains('Concentration (C)') |data_activity_blue.loc[:,'Corresponding life cycle stage'].str.contains('Agg - M+C')


0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
51    False
52    False
Name: Corresponding life cycle stage, dtype: bool

In [30]:
path1=Path('D:/Documents/philippea/OneDrive - BRGM/IMPMET/Python/Data')
name_file='EF 3.0v2.xlsx'
pathEF=path1 / name_file

name_sheet='Cut-Off AO'

In [31]:
pd.read_excel(pathEF).loc[:,'kg CO2 eq'].drop_duplicates()

KeyError: 'kg CO2 eq'

In [32]:
docEF=pd.ExcelFile(pathEF)
all_sheetsEF=docEF.sheet_names#.str.contains('PR')
#sheet_names_list = [name for name in all_sheetsEF if "PR" in name]
all_sheetsEF

['Feuil28',
 'Feuil27',
 'Feuil26',
 'Feuil25',
 'Feuil24',
 'Feuil23',
 'Feuil22',
 'Feuil21',
 'Feuil20',
 'Feuil19',
 'Feuil18',
 'Feuil17',
 'Feuil16',
 'Feuil15',
 'Feuil14',
 'Feuil13',
 'Feuil12',
 'Feuil11',
 'Feuil10',
 'Feuil9',
 'Feuil8',
 'Feuil7',
 'Feuil6',
 'Feuil5',
 'Feuil4',
 'Feuil3',
 'Feuil2',
 'Feuil1']

In [64]:
path1=Path('D:/Documents/philippea/OneDrive - BRGM/IMPMET/Python/Data')
name_file='list_country_virgule.xlsx'
pathcv=path1 / name_file

name_sheet='Cut-Off AO'
list_pays_virgule=pd.read_excel(pathcv)#.values.to_list()
list_pays_virgule=list_pays_virgule.loc[:,'Unnamed: 0'].values.tolist()

In [70]:
data=pd.read_excel(pathEF,sheet_name='Feuil4',usecols='A:C')#,column=['comp','sub','flow'])
data.columns=['comp','sub','flow']#.loc[data.iloc[:,2].str.contains('|'.join(list_pays_virgule))== False]
data

,comp,sub,flow
0,Air,(unspecified),1-Butanol
1,Air,(unspecified),1-Butene
2,Air,(unspecified),"1-Butene, 2-methyl-"
3,Air,(unspecified),"1-Butene, 3-methyl-"
4,Air,(unspecified),1-Hexene
...,...,...,...
1865,Air,(unspecified),"Toluene, 3-ethyl-"
1866,Air,(unspecified),"Toluene, 3,5-diethyl-"
1867,Air,(unspecified),"Toluene, 4-ethyl-"
1868,Air,(unspecified),Undecane


In [119]:
i=0
for feuille in all_sheetsEF:#[0:2]:
    data=pd.read_excel(pathEF,sheet_name=feuille,usecols='A:E')
    data=data.iloc[:,[0,1,2,4]]
    data.columns=['comp','sub','flow','value'+str(i)]
    data=data.loc[data.iloc[:,2].str.contains('|'.join(list_pays_virgule))== False]
    data=data.drop_duplicates()
    if i>0:
        #display(data)
        merged_data=pd.merge(merged_data,data,on=['comp','sub','flow'],how='outer')#,right_on=['comp','sub','flow'],left_on=['comp','sub','flow'],how='left')
        i+=1
    else:
        merged_data=data.copy()
        #display(merged_data)
        i+=1

In [120]:
merged_data.to_excel('merge_ei_elementary_flow.xlsx')

In [121]:
merged_data.loc[:,'flow'].drop_duplicates()

0           Aluminium
10           Antimony
20       Antimony III
30         Antimony V
41            Arsenic
             ...     
40073     Thorium-230
40074     Uranium-234
40077     Uranium-235
40080     Uranium-238
40083       Xenon-133
Name: flow, Length: 4257, dtype: object

In [125]:
merged_data[merged_data['flow']=='Aluminium']#drop_na()

,comp,sub,flow,value0,value1,value2,value3,value4,value5,value6,...,value18,value19,value20,value21,value22,value23,value24,value25,value26,value27
0,Air,(unspecified),Aluminium,1.880000e+05,NaN,NaN,NaN,NaN,NaN,2.084800e-08,...,NaN,NaN,NaN,NaN,2.084800e-08,NaN,NaN,NaN,NaN,NaN
1,Air,low. pop.,Aluminium,1.890000e+05,NaN,NaN,NaN,NaN,NaN,8.194000e-09,...,NaN,NaN,NaN,NaN,8.194000e-09,NaN,NaN,NaN,NaN,NaN
2,Air,"low. pop., long-term",Aluminium,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000e+00,...,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN
3,Air,high. pop.,Aluminium,1.870000e+05,NaN,NaN,NaN,NaN,NaN,3.350200e-08,...,NaN,NaN,NaN,NaN,3.350200e-08,NaN,NaN,NaN,NaN,NaN
4,Air,stratosphere + troposphere,Aluminium,1.890000e+05,NaN,NaN,NaN,NaN,NaN,8.194000e-09,...,NaN,NaN,NaN,NaN,8.194000e-09,NaN,NaN,NaN,NaN,NaN
5,Water,(unspecified),Aluminium,2.045000e+05,NaN,NaN,NaN,NaN,NaN,7.731500e-10,...,NaN,NaN,NaN,NaN,7.731500e-10,NaN,NaN,NaN,NaN,NaN
6,Water,"groundwater, long-term",Aluminium,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000e+00,...,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN
7,Water,ocean,Aluminium,1.430000e-15,NaN,NaN,NaN,NaN,NaN,3.001600e-13,...,NaN,NaN,NaN,NaN,3.001600e-13,NaN,NaN,NaN,NaN,NaN
8,Water,river,Aluminium,4.090000e+05,NaN,NaN,NaN,NaN,NaN,1.546000e-09,...,NaN,NaN,NaN,NaN,1.546000e-09,NaN,NaN,NaN,NaN,NaN
9,Soil,(unspecified),Aluminium,2.760000e+05,NaN,NaN,NaN,NaN,NaN,1.036700e-09,...,NaN,NaN,NaN,NaN,1.036700e-09,NaN,NaN,NaN,NaN,NaN


In [12]:
a=ecoinvent_data.loc[:,['Activity Name','Geography']].drop_duplicates()

act_name=ecoinvent_data.loc[:,'Activity Name'].drop_duplicates()
nb=0

for name in act_name:
    if len(a.

0        bark chips, wet, measured as dry mass to gener...
1                                 market for nitrous oxide
2            market for sawdust, wet, measured as dry mass
3        market for sewage sludge, 70% water, WWT, WW f...
4                                phenolic resin production
                               ...                        
21202                          market for dimethyl sulfide
21218    coffee green bean production, arabica, mechanized
21224      market for hard disk drive, for laptop computer
21229        market for storage building, chemicals, solid
21230     market for washing, drying and finishing laundry
Name: Activity Name, Length: 8278, dtype: object

In [13]:
a

,Activity Name,Geography
0,"bark chips, wet, measured as dry mass to gener...",CH
1,market for nitrous oxide,GLO
2,"market for sawdust, wet, measured as dry mass",CH
3,"market for sewage sludge, 70% water, WWT, WW f...",CH
4,phenolic resin production,RER
...,...,...
21231,market for kerosene,BR
21232,treatment of used bus,CH
21234,market for isophorondiisocyante,RoW
21236,methacrylic acid production,RER


In [14]:
act_name

0        bark chips, wet, measured as dry mass to gener...
1                                 market for nitrous oxide
2            market for sawdust, wet, measured as dry mass
3        market for sewage sludge, 70% water, WWT, WW f...
4                                phenolic resin production
                               ...                        
21202                          market for dimethyl sulfide
21218    coffee green bean production, arabica, mechanized
21224      market for hard disk drive, for laptop computer
21229        market for storage building, chemicals, solid
21230     market for washing, drying and finishing laundry
Name: Activity Name, Length: 8278, dtype: object